# In this, we will analyze the data that has been generated.

In [3]:
import pandas as pd
import numpy as np
import matplotlib as pyplot

# compute reverse complement
def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return ''.join([complement[base] for base in reversed(seq)]

## Let us first analyze the substitution kmers

In [52]:
kmers_expected = pd.read_csv('kmers_single_substitution', header=None)
kmers_actual = pd.read_csv('substitution_kmers', header=None)

kmers_expected_set = set(kmers_expected[0])
kmers_actual_set = set(kmers_actual[0])

print('Num kmers in expected: ', len(kmers_expected_set))
print('Num kmers in identified: ', len(kmers_actual_set))

Num kmers in expected:  3787
Num kmers in identified:  3219


In [29]:
# add the rev complements in each of these sets
for kmer in kmers_expected_set.copy():
    kmers_expected_set.add(reverse_complement(kmer))

for kmer in kmers_actual_set.copy():
    kmers_actual_set.add(reverse_complement(kmer))
    
missing_kmers = kmers_expected_set.copy()
for kmer in kmers_actual_set:
    try:
        missing_kmers.remove(kmer)
        missing_kmers.remove(reverse_complement(kmer))
    except KeyError:
        continue
print('Num kmers in expected but not in actual: ', len(missing_kmers))

Num kmers in expected but not in actual:  4050


In [30]:
missing_kmers2 = kmers_actual_set.copy()
for kmer in kmers_expected_set:
    try:
        missing_kmers2.remove(kmer)
        missing_kmers2.remove(reverse_complement(kmer))
    except KeyError:
        continue
print('Num kmers in identified but not in actual: ' , len(missing_kmers2))

Num kmers in identified but not in actual:  2912


### So, there is a big discrepancy!

In [8]:
# search kmers in genome

In [31]:
# let us search these kmers in our original string
with open('ndl.fasta') as f:
    genome = f.readlines()
genome = genome[1].strip()

for kmer in list(kmers_expected_set)[:10]:
    print ( genome.find(kmer), genome.find(reverse_complement(kmer)) )

66211 -1
40559 -1
-1 108544
-1 2217
-1 102750
48453 -1
-1 101504
-1 62808
-1 35083
-1 103523


In [32]:
# let us search expected kmers in the unitigs
with open('ndl.fasta_unitigs.fa') as f:
    unitigs_orig = f.readlines()
unitigs_orig = unitigs_orig[1::2]
unitigs_orig = [l.strip() for l in unitigs_orig]

for kmer in list(kmers_expected_set)[:10]:
    found_this_kmer = False
    for i, unitig in enumerate(unitigs_orig):
        if kmer in unitig or reverse_complement(kmer) in unitig:
            print(kmer, ' found in ', unitig, ' which is ', i, '-th unitig')
            found_this_kmer = True
    if not found_this_kmer:
        print(kmer, ' not found!')

ATTTTGGGTATTTAAAAAATA  found in  AAAAAAATAAAAAAAAATAATTACATAAGGAGGGGTAGATGGTGATGAGAAATTTCAAAAAGAAAAGAATGTGGTTTACACATATAAAAATAAAGCCCATATATGATTTTCCTACTGGGTATAGGTTTAAATATTTTAAACTTTATTTTATTTAAGTTTTTAAATATTTTATATATTTGTAAAAATTTTTTTTATGATTCTTTATTTAATATCAATTTTTTTAATTATTGTTTTTTTATTAATAAAAAAATTTTGATTGGAAAAAAGCATAATCAATTATCTATTTTTAAAAAAACAATATATATTAATTATTTTATTTCTTTTTATAAATTTTTTATTTTTTCCAGAGTAAAAAGTGGAGATATTAATTTTTTTAGTAGAATAAATAATGAAATATTATTTTTAGCCTCTTTTAAAATATTTTTTTTAATACTTAGTAATATAACAATACCCCTCTATTCTTTAAATTATTTTAATTTCTTTTTGACATACAGATTTAATTTTTCTAATATAATATTTTTCATATATTTAAATAATTTAAATGTTTTTAAATTTTATAAATGTTTTTTAAATAAAACAATTATATATTATATGATAAATAATAATATATTTTATTTTTTGCAATCTTTAAATGATTTTAAATTATATTATATTTTTTTTTCAGAATTATCAAGTTTTTATTCAGAACATATATTCTTTTTTAATTCAAAAGATTTATATTTAGATATATTATCTAACTTTTTATTTTTTAAATACCCAAAATATATAATTATAAATAATTTTTTTAATTTT  which is  35 -th unitig
AATAACAAAAATATAATTTGG  found in  AAAAATAAATTTTTTAAATATTTTATTGACATTTCAAAATGAATAAAAAATTAATAATTAATGAAATGTTATCAAATAACATACATTTAGGTCATAAAAAAAATTTATTAAACAAG

# So, we have found all the kmers in genome, and in some unitig.

## <span style="color:red">Now, let us see what these unitigs are aligned to</span>

In [38]:
# first, read unitigs from the alignment
with open('alignments') as f:
    alignments = f.readlines()
    
unitigs_in_alignment = alignments[::7]
unitigs_in_alignment = [line.strip() for line in unitigs_in_alignment]

print('First unitig: ', unitigs_in_alignment[0])
print('Last unitig: ', unitigs_in_alignment[-1])

First unitig:  TTAAAAAATTTTTAAAAAAATTTTGTAAACTAAAATAAAATATAAAATAAAAAAAATCATTATGTTTTAAAATATTTTTTTTAAAAAA
Last unitig:  AAATAAATAATTTAAAAAAAATAATTGGTTCTAGTTTTAATAAAAATAATAAAAAAAAAGCATTTAAAAGATATAAGATTTTAAAAACTTTTTATAATTTTAATATAAATCCTTTAAATATATTTTTAAATAATCTATTGGTTTTACCTGCGGATTTAAGGCCCATAATAAGATTACCTAACGATACTGTCGTTTCTTCAGATTTGAATGAGTTATATAAGAGAATTATAAATAGTAATAATAGACATAGAAAAATAAATAAAATTTATAGGATACCCTCAAATATAATATTAAAAGAAAATAAAATTTTACAAAACCACATAGATAATTTGTTTGTAGGTAATCATAAAAAAAGTAAATTCAAATCATTATTAGAAAATTTAGAAGGAAAATTAGGTTTTTTTAGAAAAAATTTATTAGGAAAAAGAGTTGACTACTCTGGAAGGGCTGTTATAACTTCAGGACCTAATATGAAATTTGGGGAATGTGGTCTACCAATTATAATAGCTATGGAAATATTTAGACCCTTCATATATAATAAAATTTTACTTGATAATTCTTTTAATATAAAGATGGCGAAAAAGGAAGTGGATAAAAATACAAAATTATCAAAGTCTATATTAATAGATTTATTTAGAGATTTTTATATAATTTTAAATAGAGCTCCTACACTACATAGATTGGGAATACAAGCATTTAAACCTATTCTTACTGATGATAATGTTATAAAAGTGCACCCCTTGGTTTGTGCAGCCTATAATGCAGACTTTGATGGTGATCAAATGGCAGTTCATATGCCAATATCTTTAAAATCTAAATTAGAAATTAAAAAATTATTAAACTGTAAAAATAATATATTATACCCTTCAAATGGAACTTGTT

In [39]:
# now, check if first 10 unitigs in "ndl.fasta_unitigs.fa" match these unitigs
print('Num unitigs in cuttlefish output: ', len(unitigs_orig))
print('Num unitigs in alignment  output: ', len(unitigs_in_alignment))

#print('First five unitigs in cuttlefish output:')
#print(unitigs_orig[:5])

#print('First five unitigs in alignment output:')
#print(unitigs_in_alignment[:5])

for unitig_in_cuttlefish_output in unitigs_orig[:10]:
    found = False
    for unitig_in_alignment in unitigs_in_alignment:
        if unitig_in_cuttlefish_output == reverse_complement(unitig_in_alignment):
            found = True
            break
        if unitig_in_cuttlefish_output == unitig_in_alignment:
            found = True
            break
    print(found)
    if not found:
        print(unitig_in_cuttlefish_output)

Num unitigs in cuttlefish output:  510
Num unitigs in alignment  output:  510
True
True
True
True
True
True
True
True
True
True


# Weird: some are found, some are not! We gotta check why some are not found.

# Edit: now all unitigs in cuttlefish output is found in alignment file

In [26]:
# Next plans: we have to first check that we can look up any alignment-unitig in the cuttlefish-output.
# Then, for all expected kmers that are missing in the actual output, we will look at the unitig it is in, 
# and then manually check its alignment

## Let us first check the lengths of the unitigs in both sets

In [40]:
unitigs_cuttlefish = unitigs_orig
lengths_unitigs_orig = [ len(utg) for utg in unitigs_cuttlefish ]
lengths_unitigs_orig.sort()
print('Sorted lengths of the unitigs in cuttlefish output: ')
print(lengths_unitigs_orig)

length_unitigs_in_alignment = [ len(utg) for utg in unitigs_in_alignment ]
length_unitigs_in_alignment.sort()
print('Sorted length of the unitigs in the alignment file: ')
print(length_unitigs_in_alignment)

Sorted lengths of the unitigs in cuttlefish output: 
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 27

In [42]:
# differences of the two sets of unitigs
print('In cuttlefish, not in alignment file')
print(set(unitigs_cuttlefish) - set(unitigs_in_alignment))

print('In alignment file, not in cuttlefish')
print(set(unitigs_in_alignment) - set(unitigs_cuttlefish))

In cuttlefish, not in alignment file
set()
In alignment file, not in cuttlefish
set()


In [43]:
# now, investigate the kmers that are expected to be found, but not really identified
kmers_expected_but_not_identified = kmers_expected_set - kmers_actual_set

unique_kmers_expected_but_not_found = set()
for kmer in kmers_expected_but_not_identified:
    kmer_rc = reverse_complement(kmer)
    if kmer < kmer_rc:
        unique_kmers_expected_but_not_found.add(kmer)
    else:
        unique_kmers_expected_but_not_found.add(kmer_rc)

print('Num of kmers we are investigating: ', len(unique_kmers_expected_but_not_found))

Num of kmers we are investigating:  2025


In [44]:
kmer_to_unitig = {}
for kmer in unique_kmers_expected_but_not_found:
    # locate the unitig it is in
    for unitig in unitigs_in_alignment:
        if kmer in unitig or reverse_complement(kmer) in unitig:
            kmer_to_unitig[kmer] = unitig
            
                

ATATGTATATAATTTTTTATC AAGGGATTCGAACCCTTGATAAAAAATTATATACATATTTTCTAAATATGTGGTTTAAACCTCTCACCCACTTCTCTATATTATTATTTTAGTATTTAGTGAATTAAAAAATTTTTTAAAA
TAAAAATTATGAATTATATAA AATATATTATTTATATAATATCTATAAGATAAATTTATTTTATAATTAACACTATGAATCAATATTATAGTGCTATAATTTTTATATAATTCATAATTTTTAATTATTGATAAAAAAGGTGAAAGCCCTGTTCCTGTTGATAAAAGCCATAGATTTTTTCCCTTATTTAAAAGGTTTTCAAAAATTAAAGTTCCAGAAGGTTTTTTTGATATATATATATTATCTCCTACGATTAAATTTTTTAAGATATTAGTAAAAATTCCATTTTTAATTTTTATACTTAAAAATTCTAAATAATTATCATAATTAGAACTTAATATACTATAAGCTCTGTATATAATTTTATTTTCTTTATCTATTAAACTTAAAAGCACAAATTCACCATTTTTAAATTTAAAATTATTTTTTCTTGTGCATTTAAAACTAAATAGGTTTTCATTTCAGTGATGGACACTTTTTATTTTTTCTAAATTATATTTTTCCATTTATAATGAAAAATATAATAATTGTTTTTTTATCTGGAGGAATAGATTCTTTATTTTCCATATTAATTTTAAAAATCTTAAAATATAGATTATATGGTTTTTATATGTTAAATTGAATAATATTAAAAAAAGAAAAAATTATAAACATAATAATTAACATATCTAAAATACTTAATATAAAAATAATAATTTTTAATTTTAAAAAGATATATTTTAGAAAAATCTTTTTAAAATTTTTAAAAAA
AAATATATTAGTTTCTTAAAT AAAAAACACAGTGCTCTAATTGATGTTTTTTTGTTAAAAAAAGTTTACTATGAACTTTATAAAAAATATAATAAAATAAAGT

TTATTATTATCAAATAATAAA AAAAATAATTTTTTATTTAAAAAAAATAGTTTAATAGATTTAAATATACTACATAAAATTAAGAAGCTAAACATAAATACATTAATTGTGAGATCTCCATTAAAATGTAAATCTAAATATGGCATATGTTGCCTTTGCTATGGTGTCGATTTAGGAAAAAATGAAATGGTTTCTATAGGAGAAATGGTTGGTATTGTTGCAGCCCAATCTATTGGAGAACCTGGAACTCAATTAACAATGAGAACATTTCATACAGGAGGATTTATAAGTAAAGCTAAAAAAAACACTTTTTTTAAATCAAATTATTCAGGATTTTTAAATTATTCTTACAATTTAAAATATGTAACAAATAAATATAATTTAAAAATAGTTATATCTTTATCTAGTAATATAGAAATTTATTTTGATTCTAAAAAAGAAAATTTATTAGAATCTTTTAATGTTCCATACGCATCTTATTTATTATTTGATAATAATAAAGAAATAATTAAAGGTGAAATAATTTGTAAATGAGATTCTGATTATTCTTTAATAATTTCAGAATTTAGCGGCAATATAAAATATAAAAATTTAAAAGATGATAATTATTTAAAAAGAAAGGATTTAAATATTTATGAAATTAAAAACAATAAAAATATGCCCTTTATACAAATATTAAATGATGGGGTGGAGTTATTTAATTATAAATTGAGAGGAGGCGATCTTTTAATGTTTGAAGAAGGCGATTTTGTGAATGTTGGAGATGTGTTAGTAAAACACCCTTTAGATTTTAGTATAGCTGAAGATATAATAGGTGGTTTACCTAAAATAGATAATTTTTTTGAAGCTAGATATCCTAAAAATAGCGCTTTAATATCTCCAGTTTCAGGATTTATTAAATTTTCATTTTTAAAAAAAAAAAAGTGTATTAATGTATATATTAAAAATAAAATAATTT
AATTTAAATAACTTTTTAATA TTATAAAAAAAATAATTT

AAAGAATCTATTATATCAAAA AAAAATATATTTTTTATAAAAAATAAAAAAAAATCTAAGTTAATATTTTTTAAAAAAGTTCAGAACAAACTAAGAAAAATATGTTTTCAAGATTTTAATTTTAGAAATAATATAAATGAAGATGTTCATAAAAATTATTTTAAAAAAAATTTTTTAACAACAAATATAGGTTCTTTTCCTCAAACTAGAAAAATAAGGAAAATAAGATCTGATTTTAAAAATAATTTAATAGAAAAAATTTTTTATGAAAAGTTAATAAAAAGAGAGATTATTTATATATTAAAAAAACAATTAGATTACAGAATAGATGTTTTAGTTCATGGAGAACCTGAAAGGAATGATATGGTTGAATATTTTGCTGAAATTATAAACGGTTATGCTATAACAAAAAATGGTTGGGTTCAAAGTTATGGTTCAAGATATGTTAAACCACCTATTATATATGGCGATTTATTTTTAAAATCATACATGACAATAAACTGAATTAAATTTTCACAAAAATTTATAAAAAAACCTTTAAAAGGGATGTTGACAGGTCCTTTAACTATGATTAAATGATCTTTTATAAGAAATGATCAACCATTTTATATAACAGCTATACAAATATCAATTATATTGAATAAAGAAATTTTAAAATTAGAAAGATTAGGGATAAAAATTATACAAATTGATGAACCCGCTATAAGAGAATTTTTACCTATAAAAAAAAATAAATATAATAAATATTTTAATTGATCTTCAAAATCATTTTTATTAATTTATAAAAATTTAAAAAATACAACACAAATACATACCCATGTTTGTTATTCAGATTTTAAAAATATAATTAATTTAATAGATAATTTAAATGTTGATGTAATATCTATAGAGACTGCAAGATCAAATTTTGATATAATAGATTCTTTAAAAAGATATAAAAGAAGTGTTGGTTTGGGTGTTTATGATATACATACCAATAGAAAAATGAGTATAGATAAA

AAATATTTTTAAAAATAAACT AAATTTAAAAAAATAAAAAATATTTTTAAAAATAAACTAATAACCAAAAAAATAATAAAAATTTTTAAAAAAATAGAATTAAAAAATATTGTAGAATTTTACAGTGGAAATAATTATATTATAAACTCATTGTTAAAAATACAATATTTTTTAAAATTAATATATATAGAAAAATTTAATAAATTAAAAAAAAAAAATAATTTTTTATT
AATATTTCAAATTTTAAAAAA AAATTTAAAAAAATAAAAAAAAATATAAGATTATATTTAAAAAATGAAGGTTGTAAAAATTATAAATATCTATTTTATAAATCCAAAAAAATTAAACCAAATGATTTTTTAATAATAAAAAATAAAAAAAGTATTTTGATTAATATTTTATTTATTAAATATTTAAATCAAATATATATAAAATTAAAAAATAATTCTTTAAACATTAAAAATTTAAATATTAAAAATTTATGTAATTGTAGATTTTCTTTTAAAATATAATAAGGTTTATAAAATTTGTAAAATTTATAAATTGTAAAATATTTTTATAATAAATAATGTTAAAATTAAAAAACTTATAAGTTAAAATTTTAATATTATTTGAAATAAGTAAAAAATTTATAAAAATTAAATTTAAATATATATATAAATTTGAAATCAAATAAAAAAAATTTATACATATTAAATCATAATGTAAAGTTTTATTTTTTAATATAACACCAAATATTAAAAATACATCAAAGATATTACTCAATAATAAAATTTTTATTAAAAAATATGTTTCTATGCTACCTTTAATTAAAATTATTAAAAATTTACTTTTTAAAAAACCAAAATTTAAAATTTTTCTATAAAAAATATTTAAAAATATTTTACTAAAAATTTTATTAAAATTAGAAATAATCAATAAGAATTTTATTATTAAGTAATATTTTATTATATAATTATATAATAGTATCATTAATAACACTTAAAGTGAAAAATAT

AAAATAAATATAATTGAAATT AAAATTAAAAAATAAATTAAAAAAAAATATTAATTTAAATTTATGATTTATTAAATTGTTATAATTATATTATTAAAAACAAAATCAAAAATTTGGCAAGAATTAAAATATTTTTTTCTAATAAAAAAAATTCCCTCATTTATTTTTATTTTACAATAATTACAAATATTAAAACCGTTGCAATTATTTTTAATTTCTAAATTATTTTCAATTAATATTTTACATATACTTATATTTTTTTTAAAATATCTAATTATTTTACCATTATAACAACTATTTTTATTAGCTAAAATTATTATTTTTTTCATTAAAAAAATTTAATAATATAATTATTTTTATAGTTAGATAAACAAAAATAAAGTATTTCTTCAAATAAAATTGAACAATGATATTTTATGATTGGTAATTTCAATTTTTTTGATATATAATTGTTTTTTAGTAAAGGGATTTTATAATAAAATCTATTTATTGTATAAAAAGATCCTAAAAATATTGCAGATAAGGCAGAGCCACACCCATATATTTTGTTTTTAAAATTTATTACTGACAAATTAAAATTATAAAAAACTAAACAGATAGAAAATACATCTCCGCAATAATAATCTCCAAGGCTCCCTTCTCCCGAATTATTTAAGTTTTTAAAATAAATATAATTGAAATTTTTATAAATACTAAAAAAAAATAAAATTTTGTTATTATATGACATTTAAAAACGGAGAAATCAATCTTAATCTATATACACTATTGAATAACAATTTTAAAAGAAAATATAAATCTGTTATTTTATTAAATCTACCAAATGTGAACCTTATTGAACCATTTAAAGGAATAAATTTATTTAATGATTTTAAAACATAAGAACTTTCTAAAATACTTGAAGAGCATGCAGAACCGGTTGAAACAGCAATATTTTTTATTGATATTATTAAAGATTCACCTTCTATAAAATTAAAACTTATATTCAAATTATGAGGTATC

ATTTGGTAGCTAAAAATAGTG ATTTAATAAATATTTTAAAGTTAATAATTTCTTTTAAAGATGAATTATTTAAAGAAAGCGATAAACTAAGTTTATCAAACCTAAATTTAATTATGATTGGTGACACTTTTGAAAATATTTTTTTAAATTTTTTTTTAAAATTAAAAAGTAGCGTAGAAAATGGTTTTTTTGATATTGGTAATAATTATACTTTAAATTCAGTTTTAAAAATATATATTCTAGTTAATGAAATAAAAGATTTTTTTAATATGTCTAAATATTGTCAATATTTAGATAAAGTAAATATTCTTTCAGAAATATCTCACAAAAGAAGAGTTACATTTCTTGGTGTTATAGGAATATCTAAAAAATACTCTGGTTTTGATTCTAGAGATCTAAATTCAAGTCACTATTCAAGAGTATGTCCTATTGAAACTCCAGAGGGTATAAACATAGGTCTAGTTAATTCTCTAGCTCTTTTATGTAGTGTGGATAAATATAAATTTTTAATAGCTCCTTATTATAAAATTATTAATGGAAAAATTACTAAAAAATGAAAGTATTTATCTGTAGAGGAAGAATTTAATTCTTTTATTGCAGAACCAACAATTATGAATTATTTTGAATCTTTAGAAAGTATTGATTTAATTTTAGCTAAATATGGCGAGCAAATATTATTCACTAATTTTAAACTTATTAAGTATATAAATATTTTACCTTATCAAATGCTTTCAATATCAACCATTTTAATTCCTTTTTTAGAAAATAATGATTCAAATAGAGCGTTGATGGCTTCTAATATGTTAAGACAAGCTTTACCTTTATCTAATCCAGAAAAACCAAAAATATCTACAGGCTTTGAAAATTTGGTAGCTAAAAATAGTGGTACAATGCTGATTTGTAAGAATAAAGGCATTTATAATTATTCAGATTCTAGATTAATAATATTGAAAAATTATTTTATTTTTAA
GAAAATTTAAATATATTGGAA ATAAAT

ACACAACGCGCCCTTCGATAT ATTAAATTTTAAAATATTTAATTCCTTATTTAAAAAATTTATATCGAAGGGCGCGTTGTGTATAATTAATTTAGAATTATATATAAAATTTAAAAATTTA
AATTTATTAAATATATTTAAA AAAAATTTTTAAAAAATATAAATTAAAAAAATGTGTGTGGCGAAAAGGGATACGCGGTAGGTTTAGAACCTGTTAACTATTTTTAGTTTTGTGGGTTCAATTCCCACCACACATAAAATATTTAAAAATTTTCTCCCTAACCTGGGGTTGAACCAGGGACCGATGGATTAACAATCCATTGCTCTACCTTCTGAGCTATTAGGGAATTTAAATTTTTATTATATGAAGTTTATTTTTAATTTTTTCTCATTTTTTTTTAAAATTATATTTTTTATAATTTTTAATATTTGGTCATATTTTTGATAAATTACAATTTATTTTTATAAATATTTTTTGATCAATATTTAAGTCATATTCAGAATATATGGCATTTGTTGGACACTCAGGAACACATACTGCACAATCTATACACTCTAAAGGATTTATAACCAAAAGGTTTTCACCCTCATAAAAACATTCAACAGGACAAACTTCTACACAATCTGTGAATTTACATTTTATACAGTTTTCTGTTACTACATAACTCATGGGTGTTCAAAAAAGTAAAAAATCTTTATCAAAAAAAAAGAGTAGGATAAATTCTAAAATTTTTAAAAAAAATATTAATTTTTTTATAAGTAAAGAAAATAATTTAATTAAATTAAGTCATCATAAGCAATATAAATAAGTTATATGAATTTATTTTTTCCGGAGTGTGGCGTAGATGGATAGCGCACCTGATTTGGGTTCAGGGGGTCATAAGTTCAAATCTTATCACTCCGAAAAATATTTAAGATTTTTAATAAATTTTTTATAAAAAATATTTAATTTTAAAAAAGAATCATGTCCGGAGACTATATTTATTATTTTTATATT

ATACATATTTAGTCAATTATC ATTTATATAAATATAAAAAATACCATTTTCATTGATAAGAAATAATTTTAATATTTAAAAATGAATTAAAATTAAAATTAATTAAAAGAAATAATGAATAAAAAAATAAGAAAATTATAAAATAAAAAGGTATAAAAATAAAAAAAAATTCTAAAAAAAAATTTTTTAAAAGAAAATATTTATTATTTTTTATAAAGAAATAATTTAAAAAAAATATTAAAAAAAAACTAAAAATAAGAATAAAAAATAAGAATAAAAAAATAAGAAATAATATATTGAATTTATATATTTCAATGAAAAAATCAGTAATTGGGTAATTAATATTAAAATTTTTTATATAAAACATTTAAAAATGTTAAAATTAATACTTTTAAGACATGGAGAATCCCTTGGAAATTTAGATAATAGATTTACAGGCATATTAGACGTTGATCTAACAGAAAAAGGTTATCTAGATTCATATAAATGTGGTATAAAATTAAAAAAATTAGAACATGATTTTGATATTTGTTATACTTCAAATTTAAAAAGAGCTATAAAAACATGTTGAGCAGTTTTCGATGCTTTAAATAAAAACTATATAAAATTAATAAAAATTAAAAATTTAAATGAAAGAAATTATGGTAAAATATTGGGTATGAATAAAATAGAAAGTGTTTTAAAATTTAAAAAAAAAAATGTTGATAATTGACTAAATATGTATAATTATAAACCTCCAAAAACAGATATTTTAAATAAATATTGAAAAACATTTCAAAAAAAATATATTTTAAAAAAA
AAATGTTGATAATTGACTAAA ATTTATATAAATATAAAAAATACCATTTTCATTGATAAGAAATAATTTTAATATTTAAAAATGAATTAAAATTAAAATTAATTAAAAGAAATAATGAATAAAAAAATAAGAAAATTATAAAATAAAAAGGTATAAAAATAAAAAAAAATTCTAAAAAAAAATTTTTTAAAAGAAAAT

AATATTTAATCTTATAATTAT AAAAAATTTTTTAAAATTAATATTTAATCTTATAATTATTCCCATAAAAAAATTAAATATTTTTTTATTAAAAA
GCTATTATATTACTAAAATTA ATAAATAAAAAAATAATATTGGAGAATTTAAATTTAAATGATAACTTATGTTATTATAATTTAAAAAAGGGTAACACTATTTGTGTTTTTCAGTTAGAAAGCTTGGGAATAAGAAATGTTTTAAAAAAAATTAAACCAGATAATTTTAGTAATATAATAGCTATAATATCGCTTTACAGACCCGG
AATTATATTAATATTATTTCT TTAAAAATTTTTTTATCTTTTTTCCTAATATTAAAATTTTTAAAATTAAAAATAAGCTATTTTTAATTTTTATTTTTATTGATCTTTTATCTTTAAGAGATATAAATTTCTTCAATATTCTATAGTAAATAATTTGATAATTATCTAAAATTATAAATTTTATAGAATTATTATTTATCACAATGTATATATTTTTGTTACTTTTTTTGTTTTTAGAAATAATATTAATATAATTTAAAAAATTTATTATTAAGAATTTTGATAAATTAATTTTAAACCTAATTAAATTTTTTTTTATATTTCATATTTTTTTCTTAAAAATTTTTATTGCTTTGATGTTGTATTTTATTTTATTGTTTTTTATTTTAATTTTATTATTTTTTATTTTTAAATTAATTATATTATTATTTAAATTTTTTATTATATTAAAAAAAAAAATAAAATTAATTTTTAGTATTATATTTATTTTTTTATTAAAATTTTTAAGAAAAAAATTTATTTGAATATTTTTATCTTTATAGATAAAATACATTTTTTTTTTTATTAATATTATTAACTTATTATTTTTTTTTGAGTATACTATATTTAAAATTTTTAAAATATCTAATAAAAAATTTTTTTTTATAATTATTTTATTTATCATGAAGAGAACTTATAATCCATCGAATTTAAA

AAAAAAAAAATCTCTAAAAAT AAATTTAAAAAAATAAAAAAAAATATAAGATTATATTTAAAAAATGAAGGTTGTAAAAATTATAAATATCTATTTTATAAATCCAAAAAAATTAAACCAAATGATTTTTTAATAATAAAAAATAAAAAAAGTATTTTGATTAATATTTTATTTATTAAATATTTAAATCAAATATATATAAAATTAAAAAATAATTCTTTAAACATTAAAAATTTAAATATTAAAAATTTATGTAATTGTAGATTTTCTTTTAAAATATAATAAGGTTTATAAAATTTGTAAAATTTATAAATTGTAAAATATTTTTATAATAAATAATGTTAAAATTAAAAAACTTATAAGTTAAAATTTTAATATTATTTGAAATAAGTAAAAAATTTATAAAAATTAAATTTAAATATATATATAAATTTGAAATCAAATAAAAAAAATTTATACATATTAAATCATAATGTAAAGTTTTATTTTTTAATATAACACCAAATATTAAAAATACATCAAAGATATTACTCAATAATAAAATTTTTATTAAAAAATATGTTTCTATGCTACCTTTAATTAAAATTATTAAAAATTTACTTTTTAAAAAACCAAAATTTAAAATTTTTCTATAAAAAATATTTAAAAATATTTTACTAAAAATTTTATTAAAATTAGAAATAATCAATAAGAATTTTATTATTAAGTAATATTTTATTATATAATTATATAATAGTATCATTAATAACACTTAAAGTGAAAAATATATTTAAATCAAATTCTATATTTATAAAATCAAAAATATTATAGAATTTAATATTAGTAAAATTATAAGTAAATTTTAATAAATTAAAACATATTAAAATATAACTTTTAAAAATTAATATTTCATTTACAGTTAAACTAACTCCATTTATAGATATGGAATTTTTATAAAAAATAAAATTTTTAAATTTTTTTAAACATAATAAATATATTTTTAATATATTAAAATATAAA

In [51]:
# read alignment lines again
with open('alignments') as f:
    alignment_lines = f.readlines()
    
unitig_to_alignment = {}

for i, unitig in enumerate(unitigs_in_alignment):
    line_num = i*7
    unitig_to_alignment[unitig] = alignment_lines[i*7+1 : i*7+4]
    
for kmer in unique_kmers_expected_but_not_found:
    #print(kmer)
    utg = kmer_to_unitig[kmer]
    idx1 = utg.find(kmer)
    idx2 = utg.find( reverse_complement(kmer) )
    if idx1 < 0:
        print(idx2)
        print( reverse_complement(kmer) )
    else:
        print(idx1)
        print( kmer )
    for line in unitig_to_alignment[utg]:
        print(line)

742
TATTTTTTAAATACCCAAAAT
1 AA---AAAAATA--A---A--AAAAAAT--A-ATTA-C-ATAA--GGAGGG-GT--AGA-TGGTGA-TGAGAAATT--TCAAAAAGAA--A-----AGA-ATGTGGTTTA-C---AC-A---T--A-T-A-AA----A---A-TA--AAGCCCATA-TAT---GATTTTCCTAC-TGGGTATAG------G-TTTAAATAT-TTTAA-A-CTTTAT-T-TTATTTAA-G-T--T-TT-TA--AATAT-T-T--TA--T-AT-A-TT--TG-TAAAAATTTTTT-TT-ATGATT-CT--TTATTTAATATCAATTTTTT-TAAT--TATTGTTT---T-T--TTATTAATAAAAAAATTTTGA-TT--GGAA----AAAAGCA-TAATCAATTATCTA-T--TTTTAAAAAAACAATAT-ATATTAATTATTTTATTTCTTTTTATAAATTTTTTATTTTTTCCAGAGTAAAA-AGTGGA-GATA-TTAATTT--T-TTTAGTAG-A-AT-AAATAATGAAA-TATTATTTTTA-GCCTCT--T--TT--A-AAATATTTTTTTTAATACTTAGTAATAT-AACAATACCCCTCTAT-TCTTTAAATT-A-TTTT--AA-TTT--C--TT-TTTGACATACAGATTTAATTTTTCTAA-TATAA-TATTTTTCATATATTTAA--ATA--ATT--T-AAA--TGTTTTTA-AATTTTATAAATGTTTTTTAAAT---A-AAACA--ATTATATATTATATGATAAAT--A-A-T-A-ATA-TATTT---TATTT---TTTGCAATCTTTAAA--TGATTTTAAATTATATTATATTTTT--TTTTCAGAATTATCAAGT---T---T-TTATTCAGAA-CATATATTC-TTTTTTA--ATT--CAAAAG-ATTTA--T-A-TTTAGATATATTATCTAA-CT---T-TTTAT--TTTTTAAATACCC---A--

KeyError: 'AAATAATAGAAAAAACACAGT'

## Expected:
```
S = 3787
D = 11798
```

## localms(w, x, y, z)
```
means local alignment with
Match score        w
Mismatch score     x
Gap open score =   y
Gap extend score = z
```
```
localms(3, -1, -1, -1)
S = 3219
D = 6400
```
```
localms(10, -1, -5, -5)
S = 3232
D = 6407
```

## globalms(w, x, y, z)

```
globalms(10, -1, -5, -5)
S = 1689
D = 3173
```
```
globalms(3, -1, -1, -1)
S = 1851
D = 3576
```

## Using the largest unitigs that constitute x% of all unitigs
```
x = 0.9, globalms(3, -1, -1, -1)
S = 1776.6666666666665
D = 3601.111111111111
```

```
x = 0.9, globalms(10, -1, -5, -5)
S = 1640.0
D = 3292.2
```

```
x = 0.9, localms(3, -1, -1, -1)
S = 3114.4444444444443
D = 6445.555555555556
```

```
x = 0.9, localms(10, -1, -5, -5)
S = 3120.0
D = 6440.0
```

### Varying x
fixed localms(3, -1, -1, -1)
```
x = 1.0
S = 3219
D = 6400

x = 0.95
S = 3250.5263157894738
D = 6588.421052631579

x = 0.9
S = 3114.4444444444443
D = 6445.555555555556

x = 0.8
S = 3045.0
D = 6597.5

x = 0.5
S = 2968.0
D = 5714.0
```

In [61]:
from process_unitigs import filter_by_length_percentage

unitigs = unitigs_cuttlefish
cut_off = 0.1
unitigs_filtered = filter_by_length_percentage(unitigs, cut_off)
print(len(unitigs), len(unitigs_filtered))
l = [len(u) for u in unitigs]
a = sum(l)
print(a)
l = [len(u) for u in unitigs_filtered]
b = sum(l)
print(b)
print(b/a)

510 3
121722
12683
0.10419644764298977


In [63]:
f = open('ndl.fasta')
lines = f.readlines()
print(len(lines[1]))
f.close()

112091


In [69]:
seq_a = 'AATTTTAAAAATTTTAAAAAAATAGATAAATTAATAATTC-CTGGTCAGGGATTTAACTATAAATGAATAT--TTAATTT-TTTTAAAAATAATTTTCATAAA-TTTAT-AAATTTAATCAT-AAATAAAACTATATTAGGAATTTGTGTCGGCAAACAAGTTTTTTTTAAT-AAAAGTGAAGAGTTTGATA-GTGAAAGTCTTTT-TTTATTTAAAGGAAATATTAAAAAAT-TTATAAATAATTATTTCAAAAATAAAATTCCA-AATATTGGTTGAAAAAAAGTTTTTTTTATTAATAATCATTTTTTATTAAATAATTTAAA-TAATTCAAATAATTTTT-ATTTTTCACATAGTTATTATTTAATTCCTTTTGAAAA-TAATTATACTTT-TGGGTTAACTAAAAATAGGATATGTTTTTCTTCAA-TTTATATAAAAAATAAT-ATATTTCTTTTACAATTTCATCCTGAAAAA-A-G-CTTTAAACAAGGATTTTTAATTTTCAATAATTTTTTTAAC-TGAAA--AATATAATAAAATTAAAATGT-TAATTATTCCAGCAATAGATTTAAAAAATAATAATTGTGTAAGACTAAAAAAGGGTGATTTAAATAAATCTACTGTATTTTCAAAAAATCCTCTCATAATATTTAATAAATGATTAAATTATAATCTAAATAGAATACATATAGTTGACTTAGATGGAGCAATTTATGGAAAA-CCT-AAAAATTTATTATCATTG-AAAAACATTATAAAATACAAGAATGAAAATATAAAAATACA-AATT--GGAGGAGGTGTAAGGAATATAGAGATTATAGAATATTA-TTTTGATTTAGGTTTTTCTTATATA-GTATT-AGGAACAAAAGCTATAACTGATTTAGATTTTTTACAAAATGTTTGTAAAAAA-TATGAAGAA-AAAA-TAATTTTAAGTATAGATGTTAAAAATCTTAAATTAGC-TATAGAAGGT-TGAAAAA-ACTATTATAATATAAATTTATTTAAATTTTTAAAAGAAATAAATAATTATCCTTT-AG-AGTCTATAATATATACAGATATAAA-TAGAGATGGAACTTTAAAAGGTTTGAATATTGATAAATTTAAAAAAATATCTGAGTATTTAAAA-AAACCACTGAT-AGTAAGTGGAGGTTTTTCTTCAT-TAAAGGAT-ATAAAAAATCTTTTTATTTTACAAAATAAAAA-TTT-AATTGGAATTATATTAGGTTCTTCTATATATAAAGGAAAAATAAATTTAAATATTTTACTTAATAAAATAAATTACTTAAATAAGTATTATATATCTTAAAATTTTATGAATTTTAAAAGAATAATAGCTTGTTTAGATTTTTTTAACGGCAGAATAATAAAAGGAATAAAATTTAATAATCTTATAGATATAGGC-GATCCTTTA-GATGTAGCAGAATACTATAAT-GATAACGGCGCCGATGAAC-TTGTAATATTAGATGT-TTCTGCAAGTATAGAAAATAAAAGCCAAACATTATCTTTTATTAATAAAATA-TGT-TCTA--AAATAACAGTACCCCTAACTGTT-GGAGGT-GGCGTTAAATCTTTAGAAGATTTTAAAAATCTTTTTAATTCTGGAGCTGATAAAATAAGCATAAATTC-ACATATAATAAA-TAAA-CCGGAATTTATAAAAAAAATTAAATTCTTTTACGGATCACAATGCATTATAGCG-GCGCTTGATATTAAAAGAATTTTTATTGATAAAAATTTTATTTTTT--GAAAA-ATTTATAAAAAGTCAGGTAAAACAGATACAAAATAT-AATCT-TTTTGATTTAATA-TTAAAATTTTCAGA-TTTGGGTGTAGGT-GAATTTTTAATAA-CAAGTATAGATAGAGATGGCACAAA-ATTAGGA-TTTGATAAAGAATT-ATTAGAAAAAATATCAAATATTTCAAATCTAAACATAATAGCTTCAGGAGGTGGAGGAGGGTTAAATAGTATTTTAG-AGGTTTTGAA-GATAAAAAAT-ATTAATTCTTTGC-TT-TTA-GCAAGTATAATACATAACAAATTTTACACTATTTC-TCATATAA-AAGCTTTTTTATTGAATAGTGGAATAAATGTTAAATTTTAAAAAA-TTTAAAAAATT-TATTATAAA-TAAC-TATTTTTTTTTAAATTGAAATTACTTTAATAATTTATAT-GTAGTT-TTAAAAAATATTTTAAATA-ACAAAATAATAATGTTTGCTTTAT-TTAATTATATTTCT-ATTATAAATACAATATTTAATAAATATATTTA-TTATTATTCAAGAAGTAAAAAT-TATCTTTGATTTAAAGGTGAA-AAATCA-AAGTATTTTCAAATATTTAA-GAATTTTTATATAGATTGTGATATGGATTTTATATTAATAAATATTCTTCA-AATATATAAAATATCCTGTCATATTAATTATTTTTTTTGTAATTTTAATAAAATTTTAAATTATTAA-GAAGAGATTCCTGAGTGGTCAAAAGGAGC-AGACTGTAAATCTGTTGATAGTACTA-TCTTCGTTGGTTCAAATCCATCTCTCTTCATATAATTTTTCA-ATGTAAGTGTATCTTAACGGTAAAGTAAAAACCTTCCAA-GTTTTTTAAGAGGGTTCGATTCCCTCCACTT--ACAACTGCC--TTTTTAGCTTAATATGGTAAAGCACTT-TATTAGTAATAAAGTT-GATAGCAGTTCAAATCTGCTTTAAGGCAAATAT-ATT-T-TTTATTTCTATGGTTAAGTTAAAATTTGAAAGAAATAAACCTCATATAAATATAGGAACTATAG-GTCATGTAGATCATGGTAAAACAACTC-TTAGCGCAGCTTTAGCTACTATATTATCTAAAAAATATG-GAGGAACTGCAAA-AAATTATGAGGATATAGATGCAGCACCTGAAGAAAAGCTTAGAGGTATAACAATAAACACATCTCATATAGAATATGAAACAGAAAAAAGACATTACGCCCAT-GTGGATTGTCCTGGTCATGCTGATTATATAAAAAATATGATTAT-AGGTGCAGCTCAAATGGATGCTGCAATACTTGTGG-TTTCAGCAGTTGACGGTCCTATGCCTCAAACTAGTGA-ACACATACTATTAGCAAGACAGGTT-GGAGTACCATATATAATTGTTTATTTAAATAAATGTGATATATTG-GATGATTTTGAATTTATATCTTTAGTTGAAATGG-ATATAAAA-GATATATTAAATAAATATAACTTTTTAAATGAACATACACCTT-TTATAAA-AGGTTC-CGCAAAATTAG-CGTTAGAAGGTGATAAAAGTTATTTTGGAGAAGAATCTATTTTAAGATTATCT-AAATCTTTAGATAATTTTATACCTCTTCC-TGAAAGAGTTACAGATT-CGCCC-TTTTT-AA-TGCCTATAGAAGATGTATTCTCAATTT-CAGGAAGAGGTACAGTT-GTTACTGGTAGAATAGAGAGAGGTTCCATAAAAATAGGAAATGATTTAGAAATATTAGGAATAAGTAATAACATATTAAAAACAGTA-TGTACTGGTATT-GAAATGTTTA-ATAAAATACTAGAAGTAGGATATGCAGGGGATAATGTCGGGATTTTATTAAGAAATATTGATAAAAACAAT-GTTGAAAGAGGACAAGTTTTATCTAAAC-CAGGATCTTTAACATTACATACGGA-GTTTGAATCCAA-AGCCTATATTT-TAAAATCAGAAGAAGGGGGAAGACA-TACCCCTT-TTTTTAA-TAATTATAAA-CCCCAATTTTATTTTAGAACTACAGATGTGACAGG-AGTGGTGAAATTAAA-AGATAATAAAGAGA-TGGTTATGCCCGGCGAAGAT-ATTGATA-TTATTGTTAAATTAATATCTC-CAATTGCTGTGGAA-AA-AGGT-TTGAGATTTGCAATAAGAGAGGGTGGAAAAACAGTTGGAGCTGGAATTGTAAC-AAAAGTTTTA-TA-GTTTAATAAAA-GGATGTAGCTTAATTGGTAAAGCCTTGGTCTTCAAAATCAAAA-ATTAA-G-GGTTCAAA-TCCTTTCGTCCTTATTTTTTA-AAA-TGTTAATTTTTAAAAAAA-GAAAAGGTAAAATAAAATTAAATCTATTTCCTAGTAAAGCAAATCCCTCACC-GCC-TATAGG-GCCTACATTAGGTCAGAAAGGTTTAGACATTTCTAATTTCTGTAAAGAATTCAATAAAAAAACAAATAACTTAGATAA-GTCTTGAATA--ATACCTACAGTTATAATTTATTACTCTGA-TAAGTCTTATGATTTTTTTATAAAAAAA-CCAACAGTTTTTTTTTATT-TAAAA-AATTTTTTA'
seq_b = 'AA-TTTAAAAATTTTAAAAAAATAGATAAATTTATAATTCTCTGGTCAGGGATTTAACTAT-AATGAATATGATTAATTTGTTTTACAAATAATTTTCATAAAATTTATGAAATTTAATCATTAAATAAAACTATATTAGGAATTTGTGTCGGCAAACAAGTTTTTTTTAATCAAAAGTGAAGAGTTTGATAAGTGAAAGTCTTTTGTTTATTTAAAGG-AATATT--AAAATCTT-TAAATAATTATTTCAAAAATAAAATTCCACAATATTGGTTGAAAAAAAGTTTTTTTCATTAATAATCTTTTTTTATTAAATAATTTAAAATAATTCAAATAATTTTTTACTTTTCACATAGTTATTATTTAA-TCCTTTTGAAAAATAATTATACTTTGTGGGTTAACTAAAAATA-GATATGCTTTTCTTCAACTTTATATAAAAAATAATTATATTTCTTTTAAAATTTCATCCTCAAAAACAAGGCTTTAAACAAGGA-TTTT-ATTTTCAATAATTTTTTTAACCTGAAATTAATATAATAAAATTAAAATGTCTAATTATTCCAGCAATAGATTTAGAAAATAATAATTGTGTAAGACTAAAAAAGGGTGATTTAAATAAATCTACTGTA-TTTCAAAAAATCCTCTCATAATATTTAATAAA-GATTAAATTATAATCT-AATAGAAT-CATATAGTTGACTTCGATGGAGC-ATTTATGGAAAAACCTGAAAAATTTATTATCATTGTAAAAACATTATAAAATACAAGAATGAAAATATAAAAATACATAATTTAGGAGGAGGTGT-AGTAATATAG-GATTATAGAATATTAATTTTGATTTAGGTTTTTCTTATA-ATGTGTTTAGGAACAAAAGCT-TAACTGATTTAGATTTTTTACACAATGTTTGTAAAAAAGTATGAAGAACAAAAATAATATTAAGTATA-ATGTTAAAAATC-TAAATTAGCCTATAGAAGGTCTGAAAAATACTATTATAATATAAA-TTATTT-AA-TTTTAAAAGAAATAAATAATTATCCTTTTAGGAGTCTATAATATATACAGATATAAAATAG-GATGGAACTTTAAAAGGTTTGAATATTGATAAATTTAACAAAATATCTGAGTATTTAAAACAAACCACTGATCAGTAAGTGGAGGTTTTTCGTCATATAAAGGATGAT-AAAAATCTTTTTATTTTACAAAATAAAAAATTTCAATTGG-ATTATATTAGGATCTTCTATATA-AAA-GAAAAATAAATTTAAATATTTTACTTAATAAAATAAATTACTTAAATAAGTATTATATATCTTAAAA-TTTATGAATTTT-AAAGAATAATAGCTTGTTTAGATTTTTTTAACGGCAGAATAATAAAAGGAATAAAAGTT-ATAATCTTTTAGATATAGGCCGAT-CTTTACG-TGTAGC-GAATACTATAATTGTTAACGGCGCCGATGAACGTTGT-AT-TTAGATGTATTCTGCAAGTATAGAAACTAAAAGCCAAACATTATCTTTTATTAATAAAATAATGTGTC-AGTAAATAACAGTACCCCTAACTGTTTGGAGGTTGGCGTTAAATCTTTAGAAGATTTTAAAAATC-TTTCAATTCTGGAGATGATAAAAT-AGCATAAATTCTACATATAATAAACTAAAACCGGAATTTATAAAAAAAATTAAATTATTTTACGGATCACAATGCA-TATAGCTAGCGCTTGATATTAAAAGAA-TTTTATTGATAAAAATTTTTTTTTTTTTGAAAACATTTATAAAAAGTCAGCTAAAAC-GATA-AAAATATTAA-ATATTTTGATTTAAAAATTAAAATTTTC-TACTTTGGGTGTAGGTAGAATTTTTAATAATCAAGTATAGATAGAGATGGCACAAACATTAGGGCTTTGATGAAGAATTGATTAGAAAAAATA-CAAATATTTCAAATCT-AACATAATAG-TTCAGGAGGTGGAGGA-GGTTAAATAGTATTTTAGGAGG-TTTGAACGATAAAAAATTATTAATTCTTTGCCTTGTTAAGCAAGTATAATACATAACAAATTTTACACTATTTCATCATATAATAAGCTTTTTTATTGAATAGTGG-ATAAATGTT-AATTTTAAAAAAATTTAAAAAATTCTATTATAAAATAACATATTTTTTTTTAAATTGAAA-TACTTTAATAA-TTAT-TAGTAGTTATTAAAAAATATTTTAAATAGAC-AAATAATAATGTTTGCTTTAGATTAA-TATATTTCTTATTATAAATACAATATTTAATAAATATATTTAGTTATTATTCAAAAAGTAAAAATCTATCTTTGATTT-AAGGTGAAGAAATCATAAGTACTTTCAAATATTTAAAGAATTTTT-TATAGATTGTGATATGGATTTTATATTAATAAATATTCTTCACAATATAT-AAATATCCTGTCATATTAATTATTTTTTTTGTAATTTTAATAAAATTTTAAATTATTAAAGAAGAGATTCCTGAGTGGTCAAAAGGAGCGAGACTGTAAATCTGTTGATAGTACTAATCTTCGTTGGTTCAAATCCAT-TCTCTTCATATAATTTGTCAGATGTAAGTGTATCTTAACGGTAAAGTAAAAACCTTCCAAAGTTTTTTAAGAGGGTTCGA-TCCCTCCACTTGTACAACTGCCCCTTTTTAGCTTAATATGGTAAAGCACTTATATTAGTAATAAAGTTTGATAGCAGTTCAAATC-GCTTTAAGGCAAATATTATTCTATTTATTTCTATGGTTAAGTTAAAATTTGAAAGAAATAAACCTCATATAAATAT-GGAACTATAGCGTCATGTAGATCATGGT-AAA-AACTCCTTAGCGCA-CCTTAGCTACTATATTATCTAAAAAATATGAGAGGAACTGCAAAGAAA-TAT-AGGATATAGATGCAGCACCTTAAG-AGAGC--AGAGGTATAACAATAAACACATCTCAT-TAGAATATGAAACAGAAAAAAGACATTACGCCCATTGTGGATTG-CCTGGTCATGCTGA-TATATAAAAAATATGATTATTAGGTGCAGCTCAAATGGATGCTGCAATACTTGTGGGTTTCA-CAGTTGACGGTCCTATGCCTCAAA-TAGTGAGACA-AT-CTATTGGCAAGACAGGTTAGGAGTACC-TATATAATTGTTTATTTAAATAAATGTGATATATTGTGATGATTTTGAATTTATATCTTTAGTTG-AATGGGATATAAAAAGATAAATTAAATAAATATAACTTTTTAAATGAACATACA-CTTATTATAAACAGGTTCTCGC-AAATTAGACGTTAGAAGGTGATAAAAGTTA-TTT-GAGAA-AATCTATTTT-AGATTATCTTAAATCTTTAGATAATTTTATACCTCTTCCATGAAAGAGTTACAGATTTCGCCCCTTTTTGAAATGCCTATAGAAGATGTATTCTCAATTTTCAGGAAGAGGT-CAGTTAGTTACTGGTAGAATAGAGAGAGGTTCCATAAAAATAGGAAATGATTTAGAAATATTAGGAAT-AGTAATAACATATTAAAAACAGTAGTGTACTGGTATTTGAAATGTTTACATAAAATACTAGAAGTAGGATATGCAGGGGATAATGTCGGGATTTTA-TAAG-AATATTGATAAAATCAATAGTTGAAAGA-G-CAAGTTTTATCTAAACTCAGGATCTTTAACATTAC-TACGGAAGTTTGAATCCAACAGCCTACATTTGTAAAATCAGAAGAA-GGGGAAGACAATACCCCTTCTTTTTAAATAATTATAAAACCCCAATTTTATTTTTGAACTACAGAT-TGACAGGGAGTGGTGACA-TAAACAGATAATAAAGAGAATGGTGATGCCCGGCGAAG-TGATTGATACTTATTGTTAAATTAATATC-CTCAATTGCTGTGGAAGAATAGGTGTT-AGATTTGCAATAAGAGAGGGTGG-AAAACAGTTGGAGTTGGAATTGTAACGAAAAGTTTTAGTATGTTTAATAAAAAGGATG-AGCTTAATTGGTAAAGCCTTGGTCTTCAAAATCAAAACATTAAAGAGGTTCAAAATCCTTGCGTCCTTATTTTTTACAAAATGTTAATTTTTAAAAAAAAGAAAAGGTAAAATAAAATTGAATCTATTTCCTAGTAAAGCAAATCCC-CACCCGCCCTATAGGCGCCTACATTAGGTCAGACAGGTTTAGACATTTCTAATTTCTGTAAAGAATCCAATAAAAAAACAAATAACTTAGATAATGTCTTGAATACCATACCTACAGTTATAATTTATTACTCTGACT-AG-CTTATGATTTTTTTATAAAAAAAACCAA-AGTATTTTTTTATTATAAAACAATTTTTTA'
seq_align = '|| |||||||||||||||||||||||||||||.||||||| |||||||||||||||||||| |||||||||  ||||||| |||||.|||||||||||||||| ||||| |||||||||||| ||||||||||||||||||||||||||||||||||||||||||||||||| ||||||||||||||||||| ||||||||||||| |||||||||||| ||||||  ||||| || ||||||||||||||||||||||||||||| ||||||||||||||||||||||||||.||||||||||.||||||||||||||||||||| ||||||||||||||||| |.|||||||||||||||||||||| |||||||||||| |||||||||||| ||||||||||||||||| ||||||.|||||||||| ||||||||||||||||| ||||||||||||.|||||||||||.||||| | | ||||||||||||| |||| ||||||||||||||||||||| |||||  ||||||||||||||||||||| |||||||||||||||||||||||.||||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||||||||||||||||||||||||||| |||||||||||||||| |||||||| ||||||||||||||.|||||||| |||||||||||| ||| |||||||||||||||||| ||||||||||||||||||||||||||||||||||||||||| ||||  ||||||||||| ||.||||||| ||||||||||||||| ||||||||||||||||||||||| | ||.|| ||||||||||||| ||||||||||||||||||||||.||||||||||||||| ||||||||| |||| ||||.||||||||| |||||||||||| ||||||||| |||||||||| ||||||| |||||||||||||||| |||||| || |||||||||||||||||||||||||||| || ||||||||||||||||||||||||| ||| ||||||||||||||||||||||||||||||||||||||.||||||||||||||||||||| ||||||||||| ||||||||||||||||||.|||| |||||||| || ||||||||||||||||||||||||||||| ||| |||||| |||||||||||.||||||||||| ||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||||||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||.|| ||||||||.||||||||||| ||| ||||| | |||||| |||||||||||| |.||||||||||||||||| |||| || |||||||| |||||||||||||||||.|||||||||||||||||||||||||||||||||| ||| || |  ||||||||||||||||||||||| |||||| ||||||||||||||||||||||||||||||| |||.||||||||||.||||||||| ||||||||||| |||||||||||| |||| ||||||||||||||||||||||||||.||||||||||||||||||| ||||||. ||||||||||||||||||| |||||||||||||||||||.||||||  ||||| ||||||||||||||||.|||||| |||| ||||||| || .| |||||||||||.| ||||||||||| .| ||||||||||||| ||||||||||||| ||||||||||||||||||||||||| ||||||. ||||||.||||||| ||||||||||||| |||||||||||||||| |||||||||| |||||||||||||||| |||||||||||||||||| ||| |||||| |||||||||| ||||||||||||| || ||| ||||||||||||||||||||||||||||||||||| |||||||| |||||||||||||||||||||| ||||||||| |||||||||||| ||||||||||| ||||||||| |||| |||||||||||||||||||| ||||||||||| |||| | |||||| ||||||||||||||||||| || ||||||||||||||||||||. |||| ||||||||| |||||||||||||||||||||||||||||||| |||||||||||.|||||||||| |||||||||||| |||||||| |||||| |||||.|||||||||||||| |||||||| |||||||||||||||||||||||||||||||||||||||||| ||||||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| ||||||||||||||||||||||||||||| |||||||||||||||||||||||||| ||||||||||||||||||||| ||||||||||||||||.||| ||||||||||||||||||||||||||||||||||||||| ||||||||||||||||||| |||||||||||  |||||||||  ||||||||||||||||||||||||||| |||||||||||||||| |||||||||||||||| |||||||||||||||| ||| | ||||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||||| ||||||||||||||||| ||| ||||| |||||||| |.|||||||||||||||||||||||||||| ||||||||||||| ||| ||| ||||||||||||||||||||.||| |.|||  ||||||||||||||||||||||||||| ||||||||||||||||||||||||||||||||||| |||||||| |||||||||||||| |||||||||||||||||||| ||||||||||||||||||||||||||||||||||| ||||| |||||||||||||||||||||||| |||||| ||| || |||||.|||||||||||| |||||||| |||||||||||||||||||||||||||||||||||| |||||||||||||||||||||||||||| ||||| |||||||| ||||.|||||||||||||||||||||||||||||||||| ||| ||||||| |||||| ||| ||||||| |||||||||||||||||||||| ||| ||||| |||||||||| ||||||||| ||||||||||||||||||||||||||||| ||||||||||||||||| ||||| ||||| || ||||||||||||||||||||||||||| ||||||||||| ||||| |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||||||||||||||||||| |||||||||||| |||||||||| ||||||||||||||||||||||||||||||||||||||||||||||| |||| |||||||||||||.|||| ||||||||| | |||||||||||||||| |||||||||||||||||| |||||| |||||||||||| ||||||.|||| |||||||||||||| |||||||||| |||||||| ||||||| |||||||||| |||||||||||||||.||||||||||| ||||||| ||||||||.| |||| |||||||||||||| ||||.||||||||||||| | ||||||| ||||||||||||||||||| | |||||||||||||| || |||| || ||||||||||||||||||||||| |||||||||||||.|||||||||||| |||||||||| || ||||||||||| ||||| ||||||||||||||||||||||||||||||||||||| ||||| | |||||||| |||||.||||||||||||||| ||| |||||||||||||||||| |||||||||||||||||||.||||||||||||||||||||||||||| |||| ||| |||||| |||||||||||||||||.||||||||||||||||||||||||||||||||.||||||||||||||||||||||||||| ||||||||||  ||||||||||||||||||||||||||||| | || ||||||||||||||||||||||| |||| |||.|||||||||| ||||| |||||||||'

k = 21
S, D = 0, 0
for i in range(len(seq_align) - k + 1):
    k_chars = seq_align[i:i+k]
    counter_space = 0
    counter_dot = 0
    for ch in k_chars:
        if ch == ' ':
            counter_space += 1
        if ch == '.':
            counter_dot += 1
    if counter_dot == 0 and counter_space == 1:
        D += 1
    if counter_dot == 1 and counter_space == 0:
        S += 1
    
print(S, D)

242 1231


In [80]:
seq_a_cleaned = seq_a.replace('-', '')
len(seq_a_cleaned)

4209

In [81]:
112071*242/4209

6443.616535994298

In [ ]:
# things to watch out for

# repeated unitig
# repeats in the genome?

# use a simulated random string




# set the indel rates to zero, and check that S is being calculated correctly




# use semi global: try a semi global
# EDLIB

In [86]:
fname = 'random.fasta'
string_length = 100000
alphabet = 'ACGT'
import random
random.seed(0)
rand_str = ''.join( random.choice(alphabet) for i in range(string_length) )
f = open(fname, 'w')
f.write('> random\n')
f.write(rand_str)
f.write('\n')
f.close()

In [92]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import time 

s1 = ''.join( random.choice(alphabet) for i in range(5000) )
s2 = ''.join( random.choice(alphabet) for i in range(5000) )
t1 = time.time()
alignment = pairwise2.align.globalms(s1, s2, 3, -1, -1, -1)[0]
t2 = time.time()
print( format_alignment(*alignment) )
print(t2 - t1)

AGC-C-GTTTCCCTCGTCGCGTCTACACAGCCAC-TG-GTC-ACC-TTT-GCA--TATTC-GT-TC-A--AGTTCTGGGGCATAGGCAAATGCT-ATGAGGC--T--A-TT-G-TA----CA--AA--ACT-CTATTGACTTGC-GACATAGCTG-ACTGGTGTCATTG--GCAACG-TCT-CAG--TCCCACGTTT-T---CCT--G--ACA-GAT-ACTTCCAATC----C-----CAGATTGTG-CCTGACCGTCA--TCG-G--AAC-CGGT---C--TGG-GCTAG--CC-A-CTAGGGTTAAGTTCTCCACCC-GA-TC--TG-GATGA-GA-A-AGTCCC-ACTAG-GAAGGTCGCG-TTCGTCT-ACTAG-C-CATACTTCTAG---G-GTCCATTGT-TCTGATCTCGGAA---A--AGCT-AA-GAAT-CG-GCT-CAAA-ATCGCCTTAAGA-TG-GTC-GA-TCCTG-CGCT--TTTAC-TTGCCACTGGGCCACCTAGCTCG-TTTAT-TGA--TA-A-C-G----C--G-GCCTG-T-C-TCGGAC--A-TCTTGAGC-ACCAGAATAAAGGTTATCG-CGATAC--CTC-GC-AATCCTCTTCCAGGTCGGCT-C--TT-G-AGCAGCAGA-TGAAGCTGCAGCGATGAGATAATATTCTGGCAGCTACGGAGAAGGAGTACCCAGACCAT-TCGAGAAG-CGCTGTCATCCCTAAGGCACGAGCAA-CGCTCGTTTTGAACATGC-TAGTCCGTTCCGGATATTATCAAATGCCCGCG-TTAATCCAT-TATGGGAACTGCACTGCCCAATAAGCGGAATTTA-GGGGCCGTGATAGCAGTCTAATC-AGCAGGT-GTTCCATAGGC-G-GCA-AACCAATCGCTCATCACCGAACACGTGAATTGGAGATGA-CAGATCGTCA-TGATATTAA-CACGC--T-CAT-CTGTGCTAGATGTGCTGGCTCAGGAGC-TAAGC-AG

In [100]:
import edlib

In [212]:
q1 = "ACCTG"
q2 = "TCAAC"
t = "TCAACCTG"

r = edlib.align(q1, t, mode = "HW", task = "path")
nice = edlib.getNiceAlignment(r, q1, t)
print("\n".join(nice.values()))

ACCTG
|||||
ACCTG


In [110]:
from process_unitigs import read_genome, process_unitigs, read_unitigs

In [111]:
unitig_set_orig = read_unitigs('random.fasta_unitigs.fa')
unitig_set_mutd = read_unitigs('random_mutated.fasta_unitigs.fa')
print( len(unitig_set_orig), len(unitig_set_mutd) )

1 1


In [112]:
utg_orig = list(unitig_set_orig)[0]
utg_mut = list(unitig_set_mutd)[0]

In [162]:
chunk_length = 5000
broken_utgs_orig = [ utg_orig[i : i+chunk_length] for i in range(0, len(utg_orig), chunk_length) ]
broken_utgs_mut = [ utg_mut[i : i+chunk_length] for i in range(0, len(utg_mut), chunk_length) ]

In [159]:
print(broken_utgs_mut[:3])

['...']


In [163]:
print(len(broken_utgs_orig), len(broken_utgs_mut))

20 21


In [168]:
num_kmers_single_subst, num_kmers_single_delt = 0, 0

t1 = time.time()
for u1 in broken_utgs_orig:

    for u2 in broken_utgs_mut:
        alignment, score, st1, st2 = None, 99999999, None, None
        
        r = edlib.align(u1, u2, mode = "HW", task = "path")
        if r['editDistance'] < score:
            alignment, score, st1, st2 = r, r['editDistance'], u1, u2
        r = edlib.align(u2, u1, mode = "HW", task = "path")
        if r['editDistance'] < score:
            alignment, score, st1, st2 = r, r['editDistance'], u2, u1

        nice = edlib.getNiceAlignment(alignment, st1, st2)
        seqA, seqB = nice['query_aligned'], nice['target_aligned']
        
        assert len(seqA) == len(seqB)

        alphabet = set('ACGT')
        num_chars = len(seqA)
        in_numbers = [0 for i in range(num_chars)]
        for i in range(num_chars):
            if seqA[i] != seqB[i]:
                if seqA[i] in alphabet and seqB[i] in alphabet:
                    in_numbers[i] = 1
                else:
                    in_numbers[i] = 2

        for i in range(num_chars-k+1):
            if sum(in_numbers[i:i+k]) == 1:
                num_kmers_single_subst += 1

        in_numbers = [0 for i in range(num_chars)]
        for i in range(num_chars):
            if seqB[i] == '-' and seqA[i] in alphabet:
                in_numbers[i] = 1
            elif seqA[i] != seqB[i]:
                in_numbers[i] = 2

        for i in range(num_chars-k+1):
            if sum(in_numbers[i:i+k]) == 1:
                num_kmers_single_delt += 1

print(num_kmers_single_subst, num_kmers_single_delt)

t2 = time.time()
print(t2-t1)
# how to get the alignment score? r['editDistance']
# how to get seq1, seq2? nice[0], nice[2]

3932 13487
9.530794620513916


In [140]:
u1 = broken_utgs_orig[0]
u2 = broken_utgs_mut[0]
r = edlib.align(u2, u1, mode = "HW", task = "path")
nice = edlib.getNiceAlignment(r, u2, u1)
print("\n".join(nice.values()))
print(r['editDistance'])

GGCGCCCT-TAGCGGTTATGACGCTTGAAGTTGAGGCTTCGCATCTAGTGTACTTTAA-GTAAGATATCCGTTTCTTCAACCTTGTCGAGTACATCGACAACCTTCGTCATTGA-G-AAC-TGATGCGTCGTGCCCACGTAACTTTACAGGGGCAACTATGGCTTCACCAAGAGATGAACAACGTTCTCGCGTCTGGCTGTCCGGCTAC-GGCTCCATGTGTAGCAGAGCGGCCGAGGCAG-TTTTT-AC-CGTCGCGC-ACATGACTCTATTGATTGTGCATTTCCTGCTATTAGGTCAATCCTACACGTGCCGCCACCAAAAACTCTCATGTGCGACGTGTGAAAGTGCGTGCTAGTACGAGCTGCGTGTGATCTCGT-ACGAGATGCGGCTAAAAGACA-GGTGCTTATCACCGCTGAGGGCTAGAAGGCAGA-CAGGCTCTATACTGGATAGGGCGCCTAAGAAGGGACTAAACCTTCAGTGTG-AAC-TCGACCCCAACGGGGGACATTCGAAGCCTGTCACCACAGCAGGACTCC-ATCTAAGTCGTTCCG-GCACGAGGTAACTCTTA-CTTAACCAATATGGGCACCGTTAGGATTTGACTGGGTTCCGGCCACCGCGA-GAACA-TGT-CAGTC-TCCTGGTCACTAC-GAGTCATAGGTCGAGGGCGCCGCTATATATCTTCGGAGGTT-CTGCGGCACGGGCCCCTACCTGATGCTAATGAAA-ACTTTCCCAGTAGGGAA-TGGGCTCGAAGCGGAGTTGAGTAGGTCTTCGCAACATAAACTCCGT-CCCGTGCAGCTCGTCGCAAGTACCG-GACAAATAAAGGCTTAATTTTACAAAGGTATTTCCACGGAGCTGTCACGGCCGGGGTGCGGGCTGCCACAAGGAACGTTCGTT-GGTACCCAACCTGGCTCTTGCTTCGGTCGTAA-TAACGCTTCGACCTAGACTACAGGGGAGCTACAGTGTATAACCCTTTGGTTGGTAAC

0.3067755255255255

# Create a random string with repeats:

1. Let us first create 10 strings of length 5000
1. Then, with repetetion, sample 20 from these
1. Concat and prepare the orig genome



In [176]:
import random
random.seed(0)

string_length = 5000
alphabet = 'ACGT'

num_strings = 10
num_samples = 20

rand_str_list = []
for i in range(num_strings):
    rand_str = ''.join( random.choice(alphabet) for i in range(string_length) )
    rand_str_list.append(rand_str)
    
rand_str = ""
for i in range(num_samples):
    index = random.choice(list(range(num_strings)))
    rand_str += rand_str_list[index]

fname = "random.fasta"
f = open(fname, 'w')
f.write('> random\n')
f.write(rand_str)
f.write('\n')
f.close()

# Now let us read unitigs of these, and try to estimate S and D off of the unitigs

In [242]:
#unitig_set_orig = read_unitigs('ndl.fasta_unitigs.fa')
#unitig_set_mutd = read_unitigs('ndl_mutated.fasta_unitigs.fa')

unitig_set_orig = read_unitigs('random.fasta_unitigs.fa')
unitig_set_mutd = read_unitigs('random_mutated.fasta_unitigs.fa')

print( len(unitig_set_orig), len(unitig_set_mutd) )

26 427


Read unitigs:

# 26 unitigs in random.fasta
# 940 unitigs in random_mutated.fasta

Let us now align a pair of unitigs

In [258]:
u1 = list(unitig_set_orig)[0]
u2 = list(unitig_set_mutd)[0]

print('Length of the unitigs: ', len(u1), len(u2))

r = edlib.align(u1, u2, mode = "HW", task = "path")
print(r['editDistance'])

r = edlib.align(u2, u1, mode = "HW", task = "path")
print(r['editDistance'])

u1 = reverse_complement(u1)

r = edlib.align(u1, u2, mode = "HW", task = "path")
print(r['editDistance'])

r = edlib.align(u2, u1, mode = "HW", task = "path")
print(r['editDistance'])

Length of the unitigs:  38 731
17
693
14
693


### This shows that we need to compute all four alignments, and then take the best. Also, another thing to keep in mind: it could very well be the case that a part of a unitig u is aligned to v, and other part is to w.

### Now let us do the same for all unitigs

In [270]:
def compute_S_D_I_N(u1, unitig_set_mutd, k):
    
    num_kmers_single_subst, num_kmers_single_delt, num_kmers_no_mutation = 0, 0, 0
    num_kmers_single_insertion = 0


    for u2 in unitig_set_mutd:
        alignment, distance, st1, st2 = None, 9999999999, None, None

        r1 = edlib.align(u1, u2, mode = "HW", task = "path")
        r2 = edlib.align(u2, u1, mode = "HW", task = "path")

        u3 = reverse_complement(u1)
        r3 = edlib.align(u3, u2, mode = "HW", task = "path")
        r4 = edlib.align(u2, u3, mode = "HW", task = "path")

        for i, r in enumerate([r1, r2, r3, r4]):
            if r['editDistance'] < distance:
                alignment, distance = r, r['editDistance']
                if i == 0:
                    st1, st2 = u1, u2
                    flip = False
                elif i == 1:
                    st1, st2 = u2, u1
                    flip = True
                elif i == 2:
                    st1, st2 = u3, u2
                    flip = False
                else:
                    st1, st2 = u2, u3
                    flip = True

        nice = edlib.getNiceAlignment(alignment, st1, st2)
        seqA, seqB = nice['query_aligned'], nice['target_aligned']
        assert len(seqA) == len(seqB)

        if flip:
            seqB, seqA = seqA, seqB

        alphabet = set('ACGT')
        num_chars = len(seqA)
        in_numbers = [0 for i in range(num_chars)]
        for i in range(num_chars):
            if seqA[i] != seqB[i]:
                if seqA[i] in alphabet and seqB[i] in alphabet:
                    in_numbers[i] = 1
                else:
                    in_numbers[i] = 2

        for i in range(num_chars-k+1):
            if sum(in_numbers[i:i+k]) == 1:
                num_kmers_single_subst += 1

        in_numbers = [0 for i in range(num_chars)]
        for i in range(num_chars):
            if seqB[i] == '-' and seqA[i] in alphabet:
                in_numbers[i] = 1
            elif seqA[i] != seqB[i]:
                in_numbers[i] = 2

        for i in range(num_chars-k+1):
            if sum(in_numbers[i:i+k]) == 1:
                num_kmers_single_delt += 1
            if sum(in_numbers[i:i+k]) == 0:
                num_kmers_no_mutation += 1

        in_numbers = [0 for i in range(num_chars)]
        for i in range(num_chars):
            if seqB[i] in alphabet and seqA[i] == '-':
                in_numbers[i] = 1
            elif seqA[i] != seqB[i]:
                in_numbers[i] = 2

        for i in range(num_chars-k+1):
            if sum(in_numbers[i:i+k]) == 1:
                num_kmers_single_insertion += 1
                    
                    
    return num_kmers_single_subst, num_kmers_single_delt, num_kmers_single_insertion, num_kmers_no_mutation

In [273]:
from multiprocessing import Pool

def compute_S_D_I_N_all(unitig_set_orig, unitig_set_mutd, k):
    num_procs = 128
    pool = Pool(num_procs)
    arg_list = [(u1, unitig_set_mutd, k) for u1 in unitig_set_orig]
    results = pool.starmap(compute_S_D_I_N, arg_list)
    pool.close()
    S, D, I, N = 0, 0, 0, 0
    for S_, D_, I_, N_ in results:
        S += S_
        D += D_
        I += I_
        N += N_
    return S, D, I, N

In [277]:
def compute_S_D_I_N_single_threaded(unitig_set_orig, unitig_set_mutd, k):
    S, D, I, N = 0, 0, 0, 0
    for u1 in unitig_set_orig:
        a,b,c,d = compute_S_D_I_N(u1, unitig_set_mutd, k)
        S += a
        D += b
        I += c
        N += d
    return S, D, I, N

In [278]:
t1 = time.time()
print(compute_S_D_I_N_all(unitig_set_orig, unitig_set_mutd, k))
t2 = time.time()
print(t2-t1)
print(compute_S_D_I_N_single_threaded(unitig_set_orig, unitig_set_mutd, k))
t3 = time.time()
print(t3-t2)

(1840, 10258, 13397, 8471)
2.5488719940185547
(1840, 10258, 13397, 8471)
9.884354829788208


In [261]:
print(len(unitig_set_orig), len(unitig_set_mutd))

26 427


In [253]:
l = list(range(100))
from itertools import product
l = list(product(l, repeat=2))
for a,b in tqdm(l):
    time.sleep(0.001)

100%|██████████| 10000/10000 [00:10<00:00, 933.47it/s]
